# I/Q demodulation and envelope detection

In [3]:
import matplotlib.pyplot as plt
from scipy.signal import hilbert, butter, lfilter, bessel, filtfilt
import numpy as np
from pathlib import Path
from numpy.fft import fft, fftshift
import requests

# ======================================================================================
# Download the darkmode.mplstyle stylesheet and use it
# ======================================================================================
url = (r"https://raw.githubusercontent.com/vincentvdschaft/quartz-website/v4/figure-generation/darkmode.mplstyle")
r = requests.get(url)
with open('darkmode2.mplstyle', 'wb') as f:
    f.write(r.content)
plt.style.use('darkmode2.mplstyle')

def pulse(t, fc, tau):
    """Transmit pulse function. Returns a Gaussian pulse modulated by a sinusoid.

    Parameters
    ----------
    t : float, array-like
        The time vector.
    fc : float
        The frequency of the carrier signal.
    tau : float
        The time delay of the pulse.

    Returns
    -------
    array-like
        The transmit pulse.
    """
    sig = 1e-7
    return np.exp(-0.5 * ((t - tau) / sig) ** 2) * np.sin(2 * np.pi * fc * (t - tau))


def fft_sh(x):
    """Computes the magnitude of the FFT of a signal and shifts the zero frequency to
    the center.

    Parameters
    ----------
    x : array-like
        The input signal.

    Returns
    -------
    ndarray
        The magnitude of the FFT of the input signal.
    """
    return np.abs(fftshift(fft(x)))

Missing colon in file 'darkmode.mplstyle', line 7 ('<!DOCTYPE html>')
Missing colon in file 'darkmode.mplstyle', line 8 ('<html')
Missing colon in file 'darkmode.mplstyle', line 9 ('  lang="en"')
Missing colon in file 'darkmode.mplstyle', line 11 ('  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"')
Missing colon in file 'darkmode.mplstyle', line 12 ('  data-a11y-animated-images="system" data-a11y-link-underlines="true"')
Missing colon in file 'darkmode.mplstyle', line 13 ('  >')
Missing colon in file 'darkmode.mplstyle', line 18 ('  <head>')
Missing colon in file 'darkmode.mplstyle', line 19 ('    <meta charset="utf-8">')
Duplicate key in file 'darkmode.mplstyle', line 21 ('  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">')
Duplicate key in file 'darkmode.mplstyle', line 22 ('  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">')
Duplicate key in file 'darkmode.mplstyle', line 23 ('  <link rel="dns-prefetch" href="http

ValueError: Missing closing quote in: '          <!-- \'"` --><!-- </textarea></xmp> --></option></form><form id="query-builder-test-form" action="" accept-charset="UTF-8" method="get">\n'. If you need a double-quote inside a string, use escaping: e.g. "the " char"

In [ ]:
# Define the parameters
fc = 6e6
fs = 7 * fc
N = 512

# Generate the signal
t = np.linspace(0, N / fs, N)
x = pulse(t, fc, N / fs / 4) + np.random.randn(N) * 0.05

# Apply the Hilbert transform and obtain the analytic signal (cutting of the negative
# frequencies).
# NOTE: The hilbert function returns not the Hilbert transform but the analytic signal.
y = hilbert(x)
# Shift the signal to baseband
y = y * np.exp(-1j * t * 2 * np.pi * fc)

# Low-pass filter
filter_order = 8
normalized_cutoff = 0.2
b, a = butter(filter_order, normalized_cutoff, "low")

# Filter using a forward backward filter to prevent phase shift
y_lp = filtfilt(b, a, y)


# ======================================================================================
# Create a figure with plots of all steps in the demodulation process
# ======================================================================================

# Plot the results
fig, axes = plt.subplots(6, 1, figsize=(6, 10))

line_color = "#AA0000"
line_width = 0.5

axes[0].plot(t, x, color="#00AAAA", linewidth=line_width)
axes[0].set_title("Original signal x(t)")
axes[0].set_xlabel("time [us]")
axes[0].set_ylabel("amplitude")

fft_axis = fftshift(np.fft.fftfreq(N, 1 / fs))

axes[1].plot(fft_axis, fft_sh(x), color=line_color, linewidth=line_width)
axes[1].set_title("FFT of the signal x(t)")
axes[1].set_xlabel("frequency [Hz]")
axes[1].set_xticks(np.linspace(-fs / 2, fs / 2, 5))
axes[1].set_xticklabels(["$-1/2f_s$", "$-f_s/4$", "0", "$f_s/4$", "$f_s/2$"])

axes[2].plot(fft_axis, fft_sh(hilbert(x)), color=line_color, linewidth=line_width)
axes[2].set_title("FFT of the analytic signal of x(t)")
axes[2].set_xlabel("frequency [Hz]")
axes[2].set_xticks(np.linspace(-fs / 2, fs / 2, 5))
axes[2].set_xticklabels(["$-1/2f_s$", "$-f_s/4$", "0", "$f_s/4$", "$f_s/2$"])

axes[3].plot(fft_axis, fft_sh(y), color=line_color, linewidth=line_width)
axes[3].set_title("FFT of the analytic signal after shifting to baseband")
axes[3].set_xlabel("frequency [Hz]")
axes[3].set_xticks(np.linspace(-fs / 2, fs / 2, 5))
axes[3].set_xticklabels(["$-1/2f_s$", "$-f_s/4$", "0", "$f_s/4$", "$f_s/2$"])

axes[4].plot(fft_axis, fft_sh(y_lp), color=line_color, linewidth=line_width)
axes[4].set_title("FFT after then performing low-pass filtering")
axes[4].set_xlabel("frequency [Hz]")
axes[4].set_xticks(np.linspace(-fs / 2, fs / 2, 5))
axes[4].set_xticklabels(["$-1/2f_s$", "$-f_s/4$", "0", "$f_s/4$", "$f_s/2$"])

axes[5].plot(t, x, color="#00AAAA", linewidth=line_width, label="original signal")
axes[5].plot(
    t, np.abs(y_lp), color=line_color, linewidth=line_width, label="demodulated signal"
)
axes[5].set_title("Same plot but in time domain")

# ======================================================================================
# Make the plot black and the axis labels white
# ======================================================================================
for ax in axes:
    ax.set_facecolor("black")
    ax.tick_params(axis="x", colors="white")
    ax.tick_params(axis="y", colors="white")
    # ax.set_xlabel("frequency [Hz]")
    ax.set_ylabel("amplitude")
    # Make the labels and ticks white
    ax.xaxis.label.set_color("gray")
    ax.yaxis.label.set_color("gray")

    ax.spines["bottom"].set_color("white")
    ax.spines["left"].set_color("white")

    # Make the titles white
    ax.title.set_color("white")


# Make the figure black
fig.patch.set_facecolor("black")

plt.tight_layout()

output_dir = Path("../content/assets")
output_dir.mkdir(exist_ok=True, parents=True)

plt.savefig(output_dir/"demodulation.png", dpi=300)